<a href="https://colab.research.google.com/github/Adithyan773/Google-colab-models/blob/main/English_to_malayalam_transilation_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od

In [ ]:
od.download('https://www.kaggle.com/datasets/rajeshradhakrishnan/ulmfit-fastai/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: qwe123achu
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/rajeshradhakrishnan/ulmfit-fastai


100%|██████████| 1.08G/1.08G [00:53<00:00, 21.5MB/s]


In [ ]:
import pandas as pd

In [ ]:
bd = pd.read_csv('/content/ulmfit-fastai/en_ml.csv')

In [ ]:
df = bd.iloc[:10_000]

In [ ]:
df.head()

,en,ml
0,The passengers were people with health emergen...,"തൊഴില്‍ നഷ്ടപ്പെട്ടവര്‍, രോഗബാധിതര്‍, സന്ദര്‍ശ..."
1,"By introducing competitive bidding, Government...",പുനരുല്‍പ്പാദന ഈര്‍ജ്ജ മേഖലകളില്‍ ഗവണ്‍മെന്റ് ...
2,She does not want to continue as Chief Ministe...,മുഖ്യമന്ത്രി എന്ന പദവിയില്‍ തന്നെ കെട്ടിയിടാന്...
3,Is this a joke?\n,‘‘ഇതെപ്പിടിയാവുമോ എന്തോ?\n
4,This was announced by Minister of State for Ex...,വിദേശകാര്യ സഹമന്ത്രി വി മുരളീധരന്‍ ലോക്‌സഭയില്...


In [ ]:
import re
def preprocess_eng(text):
  text = str(text)
  text = text.lower()
  text = text.replace('""','<quote>')
  text = text.replace('\n','')
  text = re.sub(r'\d+','<number>',text)
  text = re.sub(r'[^a-zA-Z0-9\s<>\'.,!?]','',text)
  text = text.replace("?","")
  return text

In [ ]:
import unicodedata
def preprocess_mal(text):
  text = unicodedata.normalize('NFC',text)
  text = text.lower()
  text = text.replace('\n','')
  text = re.sub(r'\d+','<number>',text)
  text = re.sub('""','<quote>',text)
  text = text.replace("?","")
  text = text.replace("\u200d","")
  text = text.replace("\u200c","")
  text = text.replace("?","")
  text = text.replace("!","")
  text = text.replace(".","")
  text = ' "<start>" ' + text + ' "<end>" '
  return text

In [ ]:
df['en'] = df['en'].apply(preprocess_eng)
df['ml'] = df['ml'].apply(preprocess_mal)

<ipython-input-10-881fd9b1cbe6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['en'] = df['en'].apply(preprocess_eng)
<ipython-input-10-881fd9b1cbe6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ml'] = df['ml'].apply(preprocess_mal)


In [ ]:
df.head()

,en,ml
0,the passengers were people with health emergen...,"""<start>"" തൊഴില് നഷ്ടപ്പെട്ടവര്, രോഗബാധിതര്, ..."
1,"by introducing competitive bidding, government...","""<start>"" പുനരുല്പ്പാദന ഈര്ജ്ജ മേഖലകളില് ഗവണ്..."
2,she does not want to continue as chief minister.,"""<start>"" മുഖ്യമന്ത്രി എന്ന പദവിയില് തന്നെ കെ..."
3,is this a joke,"""<start>"" ‘‘ഇതെപ്പിടിയാവുമോ എന്തോ ""<end>"""
4,this was announced by minister of state for ex...,"""<start>"" വിദേശകാര്യ സഹമന്ത്രി വി മുരളീധരന് ല..."


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
eng_tokenizer = Tokenizer(oov_token='<UNK>')
eng_tokenizer.fit_on_texts(df['en'])

In [ ]:
mal_tokenizer = Tokenizer(oov_token='<UNK>')
mal_tokenizer.fit_on_texts(df['ml'])

In [ ]:
eng_seq=eng_tokenizer.texts_to_sequences(df['en'])
mal_seq=mal_tokenizer.texts_to_sequences(df['ml'])

In [ ]:
max_eng_len= 50
max_mal_len= 50

In [ ]:
from tensorflow.keras.utils import pad_sequences

In [ ]:
eng_padding = pad_sequences(eng_seq,max_eng_len,padding='post',truncating='post')
mal_padding = pad_sequences(mal_seq,max_mal_len,padding='post',truncating='post')

In [ ]:
latent_dim = 64
embedding_dim = 64
eng_vocab = len(eng_tokenizer.word_index) +1
mal_vocab = len(mal_tokenizer.word_index) +1

In [ ]:
from tensorflow.keras.layers import Input,LSTM,Embedding,Dense,TimeDistributed
from tensorflow.keras.models import Model

In [ ]:
#Encoder
encoder_input = Input(shape=(max_eng_len,))
encoder_embedding = Embedding(input_dim=eng_vocab,output_dim=embedding_dim)(encoder_input)
encoder_lstm1,state_h1,state_c1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.3,recurrent_dropout=0.2)(encoder_embedding)
encoder_lstm2,state_h2,state_c2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.3,recurrent_dropout=0.2)(encoder_lstm1)
encoder_lstm3,state_h3,state_c3 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.3,recurrent_dropout=0.2)(encoder_lstm2)

#Decoder
decoder_input = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=mal_vocab,output_dim=embedding_dim,trainable=True)(decoder_input)
decoder_lstm1,_,_ = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.3,recurrent_dropout=0.2)(decoder_embedding,initial_state=[state_h3,state_c3])

decoder_dense = TimeDistributed(Dense(mal_vocab,activation='softmax'))(decoder_lstm1)

model = Model([encoder_input,decoder_input],decoder_dense)


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history= model.fit([eng_padding, mal_padding[:,:-1]], mal_padding[:,1:], epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 76s 449ms/step - accuracy: 0.8037 - loss: 7.3164 - val_accuracy: 0.8405 - val_loss: 1.6087
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 55s 440ms/step - accuracy: 0.8395 - loss: 1.5318 - val_accuracy: 0.8405 - val_loss: 1.4705
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 84s 460ms/step - accuracy: 0.8488 - loss: 1.4081 - val_accuracy: 0.8609 - val_loss: 1.4433
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 79s 436ms/step - accuracy: 0.8598 - loss: 1.3828 - val_accuracy: 0.8609 - val_loss: 1.4393
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 55s 444ms/step - accuracy: 0.8592 - loss: 1.3699 - val_accuracy: 0.8609 - val_loss: 1.4381
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 81s 432ms/step - accuracy: 0.8588 - loss: 1.3558 - val_accuracy: 0.8609 - val_loss: 1.4380
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 82s 435ms/step - accuracy: 0.8605 - loss: 1.3265 - val_accuracy: 0.8614 - val_loss: 1.4408
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 83s 441ms/step - accuracy: 0.8594 - loss: 1